In [4]:
from llama_index.core import SimpleDirectoryReader
from dotenv import load_dotenv
load_dotenv()
from llama_index.core import Settings
from IPython.display import Markdown, display
from llama_index.core import StorageContext, load_index_from_storage
from google.oauth2 import service_account
from llama_index.llms.vertex import Vertex
from llama_index.embeddings.vertex import VertexTextEmbedding
import nest_asyncio
nest_asyncio.apply()

In [5]:
from llama_index.vector_stores.neo4jvector import Neo4jVectorStore
from llama_index.core import VectorStoreIndex
import os

username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
url = os.getenv("NEO4J_URI")

In [6]:
# documents = SimpleDirectoryReader(
#      input_files=["data/Graph_Retrieval-Augmented_Generation_A_Survey.pdf"]
# ).load_data()

In [7]:
filename = "credentials.json"
credentials: service_account.Credentials = (
    service_account.Credentials.from_service_account_file(filename)
)

In [8]:
llm = Vertex(
        model="gemini-pro", 
        project=credentials.project_id, credentials=credentials,
        max_tokens=8000
    )
    
Settings.llm = llm
Settings.chunk_size = 512

In [9]:
embed_model = VertexTextEmbedding(
    model_name="textembedding-gecko@003",
    project=credentials.project_id, credentials=credentials
)

Settings.embed_model = embed_model

In [10]:
# Extract graph from documents
embed_dim = 768

neo4j_vector_hybrid = Neo4jVectorStore(
    username, password, url, embed_dim, hybrid_search=True
)

storage_context = StorageContext.from_defaults(
    vector_store=neo4j_vector_hybrid, persist_dir ='./storage_hybrid_gemini'
)

# index = VectorStoreIndex.from_documents(
#     documents, storage_context=storage_context
# )

In [11]:
# index.storage_context.persist(persist_dir="./storage_hybrid_gemini")
index = load_index_from_storage(storage_context)

In [12]:
# Define retriever
retriever = index.as_retriever(
    include_text=False,  # include source text in returned nodes, default True
)
results = retriever.retrieve("What is Graph Retrieval-Augmented Generation?")
for record in results:
    print(record.text)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: 'CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN node.`text` AS text, score, node.id AS id, node {.*, `text`: Null, `embedding`: Null, id: Null } AS metadata

Graph-Based Indexing (G-Indexing). Graph-Based Indexing constitutes the initial phase of
GraphRAG, aimed at identifying or constructing a graph database Gthat aligns with downstream
tasks and establishing indices on it. The graph database can originate from public knowledge
graphs [ 2,7,91,131,138,151], graph data [ 112], or be constructed based on proprietary data
sources such as textual [ 25,43,80,160] or other forms of data [ 169]. The indexing process typi-
cally includes mapping node and edge properties, establishing pointers between connected nodes,
and organizing data to support fast traversal and retrieval operations. Indexing determines the
granularity of the subsequent retrieval stage, playing a crucial role in enhancing query efficiency.
Graph-Guided Retrieval (G-Retrieval). Following graph-based indexing, the graph-guided retrieval
phase focuses on extracting pertinent information from the graph database in response to user
queries or input. Specifically, given a user query

In [13]:
# Question answering
query_engine = index.as_query_engine(include_text=True)
response = query_engine.query("What is Graph Retrieval-Augmented Generation?")
display(Markdown(f"{response}"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: 'CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN node.`text` AS text, score, node.id AS id, node {.*, `text`: Null, `embedding`: Null, id: Null } AS metadata

Graph Retrieval-Augmented Generation (GraphRAG) is a framework that combines graph-based indexing, graph-guided retrieval, and graph-enhanced generation to answer user queries or generate responses. 

Here's a breakdown of the three phases:

**1. Graph-Based Indexing (G-Indexing):**
- This phase involves identifying or constructing a graph database that aligns with the downstream tasks.
- The graph database can be sourced from public knowledge graphs, graph data, or constructed from proprietary data sources.
- The indexing process includes mapping node and edge properties, establishing pointers between connected nodes, and organizing data for efficient retrieval.

**2. Graph-Guided Retrieval (G-Retrieval):**
- This phase focuses on extracting relevant information from the graph database in response to user queries.
- Given a user query, the retrieval stage aims to extract the most relevant elements (entities, triplets, paths, subgraphs) from the knowledge graph.
- This is formulated as an optimization problem to find the optimal retrieved graph elements that maximize the semantic similarity between the user query and the graph data.

**3. Graph-Enhanced Generation (G-Generation):**
- This phase involves synthesizing meaningful outputs or responses based on the retrieved graph data.
- This could encompass answering user queries, generating reports, or other tasks.

Overall, GraphRAG leverages the power of graph databases to enhance the retrieval and generation of information, leading to more accurate and informative responses.


In [14]:
response = query_engine.query("How to do Query-Focused Summarization (QFS)?")
display(Markdown(f"{response}"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: 'CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN node.`text` AS text, score, node.id AS id, node {.*, `text`: Null, `embedding`: Null, id: Null } AS metadata

## Query-Focused Summarization (QFS) with Graph Retrieval-Augmented Generation (GraphRAG)

Graph Retrieval-Augmented Generation (GraphRAG) is a powerful approach for Query-Focused Summarization (QFS). Here's how it works:

**1. Graph-Based Indexing (G-Indexing):**

* Construct a graph database containing relevant information for the domain of interest.
* Represent entities, concepts, and relationships as nodes and edges in the graph.
* Annotate the graph with additional information like entity types, attributes, and relationships.

**2. Graph-Guided Retrieval (G-Retrieval):**

* Given a user query, identify relevant nodes and subgraphs in the graph database.
* Utilize graph algorithms like shortest path, random walk, or community detection to retrieve information related to the query.
* Consider the query's keywords, entities, and relationships to guide the retrieval process.

**3. Graph-Enhanced Generation (G-Generation):**

* Use the retrieved graph elements as input to a text generation model.
* The model can be a neural network like a Transformer or a sequence-to-sequence model.
* The model generates a summary that is focused on the user's query and incorporates information from the retrieved graph.

**Benefits of GraphRAG for QFS:**

* **Comprehensive Information Retrieval:** Graphs capture relationships between entities, enabling the retrieval of relevant information beyond individual documents.
* **Abstraction and Summarization:** Graph data offers a concise representation of information, reducing verbosity and improving summarization quality.
* **Contextual Understanding:** GraphRAG considers the broader context within the graph structure, leading to more accurate and informative summaries.

**Additional Resources:**

* **Graph Retrieval-Augmented Generation: A Survey:** https://arxiv.org/abs/2307.07352
* **Query Graph Generation for Answering Multi-hop Complex Questions from Knowledge Bases:** https://aclanthology.org/2020.acl-main.100
* **Natural Questions: A Benchmark for Question Answering Research:** https://aclanthology.org/2019.naacl-main.355

**Note:** This is a general overview of GraphRAG for QFS. Specific implementations and techniques may vary depending on the application and dataset.